# (Extra) Imputacion de data y analisis de Valores Nulls

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
URL_DATASET:str = r"https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"

In [ ]:
_columns:list[str] = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
columns_clean:list[str] = map( lambda x: x.lower(), _columns)

df = pd.read_csv(URL_DATASET, header=None, sep=",",names=list(columns_clean))

In [ ]:
df.replace(0,np.nan, inplace=True)

In [ ]:
(df.isna().mean() * 100).round(2).sort_values(ascending=False)

In [ ]:
df.info()

## Analisiside cantidad de nulls

In [ ]:
df.describe().T

## Imputacion por la mediana
- Mediana vs Media(Promedio)

Para un proyecto de machine learning, la elección entre media, moda y mediana depende del tipo de datos y del contexto específico. Te explico cuándo usar cada una:

## **Media (Promedio)**
**Cuándo usarla:**
- Con datos numéricos continuos sin valores atípicos extremos
- Cuando la distribución es aproximadamente normal
- Para variables como edad, ingresos, temperatura, etc. (sin outliers)

**Ejemplo:** Si tienes edades como [25, 28, 30, 32, 35], la media (30) representa bien el conjunto.

## **Mediana**
**Cuándo usarla:**
- Con datos numéricos que tienen valores atípicos (outliers)
- Cuando la distribución es sesgada
- Para datos ordinales
- Es más robusta ante valores extremos

**Ejemplo:** En ingresos como [30K, 35K, 40K, 45K, 500K], la mediana (40K) es más representativa que la media (130K).

## **Moda**
**Cuándo usarla:**
- Con datos categóricos (texto, etiquetas)
- Con datos numéricos discretos donde buscas el valor más frecuente
- Para variables como género, país, categoría de producto, etc.

**Ejemplo:** En una columna de países [México, Argentina, México, Brasil, México], la moda es "México".

## **Recomendaciones prácticas:**

1. **Analiza primero la distribución** de tus datos con histogramas o boxplots
2. **Detecta outliers** antes de decidir
3. **Considera el contexto del negocio** - ¿qué medida tiene más sentido para tu problema?
4. **Prueba diferentes enfoques** y evalúa el impacto en tu modelo


In [ ]:
import matplotlib.pyplot as plt

def generar_distribuciones(df: pd.DataFrame) -> None:
    fig = plt.figure(figsize=(16, 9))
    df.hist(bins=20, figsize=(16, 9), layout=(3, 3), color='skyblue', alpha=0.7, edgecolor='black')
    plt.suptitle('Distribuciones ', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
medianas = df.median()

In [ ]:
df_imputada_mediana = df.fillna(medianas)

In [ ]:
# generar_distribuciones(df_imputada_mediana)

## Usando tecnicas mas eficientes dado a la cantidad de datos nulls

## Missing Data Imputation

Cuando tienes valores nulos (`NaN`) en un dataset, la **imputación** busca reemplazarlos con estimaciones razonables para que los modelos de machine learning o análisis estadísticos no se rompan ni sesguen.

El **objetivo** es hacerlo de una forma que mantenga las **relaciones (correlaciones)** entre variables y **minimice el sesgo**.

---

## Métodos comunes de imputación en Scikit-Learn

### A. `SimpleImputer`

**Idea:** reemplaza los valores faltantes con una **constante**, **media**, **mediana** o **moda** de la columna.

```python
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "edad": [25, np.nan, 30, 40],
    "ingreso": [50000, 60000, np.nan, 80000]
})

imputer = SimpleImputer(strategy='median')
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
```

* **Ventajas:** simple, rápido, útil cuando los datos son poco correlacionados.
* **Desventajas:** ignora la relación entre columnas; cada columna se imputa de forma independiente.

>  Si tus variables están correlacionadas (por ejemplo, edad e ingreso), este método puede introducir sesgos.

---

### B. `KNNImputer`

**Idea:** usa los **vecinos más cercanos** (por distancia euclídea entre filas completas) para imputar los valores faltantes.
Si dos filas son similares en otras variables, se asume que el valor faltante también será similar.

```python
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
```

*  **Ventajas:**

  * Tiene en cuenta **correlaciones implícitas** entre columnas.
  * Es **no paramétrico** (no asume distribución).
*  **Desventajas:**

  * Costoso en datasets grandes.
  * Puede ser inestable si hay ruido o si los vecinos no son representativos.

> 💡 Si tus variables tienen una correlación significativa, este método suele funcionar mejor que `SimpleImputer`.

---

### C. `IterativeImputer` (basado en MICE)

**Idea:** este es el método más avanzado — usa un **modelo iterativo** para predecir los valores faltantes de cada columna **en función de las demás**.

El algoritmo más común es **MICE (Multiple Imputation by Chained Equations)**:

1. Inicializa los `NaN` (por ejemplo, con medias).
2. Toma una columna con valores faltantes y la trata como “variable objetivo”.
3. Usa las otras columnas como features para entrenar un modelo (por ejemplo, regresión lineal o `BayesianRidge`).
4. Predice los valores faltantes y los reemplaza.
5. Repite para cada columna con nulos, y hace múltiples iteraciones hasta converger.

```python
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=42)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
```

*  **Ventajas:**

  * Captura la **correlación entre variables**.
  * Proporciona imputaciones más **coherentes y realistas**.
  * Compatible con modelos diferentes al lineal (puedes usar árboles, etc.).
*  **Desventajas:**

  * Más lento.
  * Supone que las relaciones entre columnas son **predecibles y estables**.

> 💡 Si tus columnas tienen correlación significativa (edad ↔ ingreso, educación ↔ gasto, etc.), **IterativeImputer** es el más adecuado.

---

## 3. Imputación múltiple (`Multiple Imputation`)

**IterativeImputer ≠ Multiple Imputation**, aunque MICE nació de ese concepto.

### 🔍 Idea:

La imputación múltiple reconoce que **hay incertidumbre** en los valores imputados.
En lugar de generar una sola versión del dataset imputado, genera **múltiples datasets** con distintas imputaciones plausibles.

1. Se crean **m** datasets con imputaciones diferentes (por ejemplo, 5 versiones).
2. Se entrena el modelo en cada dataset.
3. Se combinan los resultados (por ejemplo, promediando los coeficientes o predicciones).

Esto permite:

* Capturar la **variabilidad debida a la imputación**.
* Evitar sobreconfianza en los valores estimados.

En Python puedes hacerlo con **`statsmodels`**, **`miceforest`** o **`autoimpute`**.

Ejemplo usando [`miceforest`](https://pypi.org/project/miceforest/):

```python
import miceforest as mf
import pandas as pd

# Creamos el kernel (modelo base para imputación múltiple)
kernel = mf.ImputationKernel(df, save_all_iterations=True, random_state=42)

# Ejecutamos el proceso MICE (crea múltiples datasets)
kernel.mice(5)

# Recuperamos las versiones imputadas
imputed_datasets = [kernel.complete_data(i) for i in range(5)]
```

* ✅ **Ventajas:**

  * Captura la **incertidumbre** en la imputación.
  * Mejores inferencias estadísticas.
* **Desventajas:**

  * Más complejo de implementar y combinar.
  * No siempre necesario para tareas puramente predictivas.

---

## 4. ¿Cuándo usar cada uno?

| Método                         | Usa correlación | Nivel de complejidad | Mejor para                                       |
| :----------------------------- | :-------------: | :------------------: | :----------------------------------------------- |
| **SimpleImputer**              |        ❌        |         Bajo         | Variables independientes o sin mucha correlación |
| **KNNImputer**                 |  ✅ (implícita)  |         Medio        | Datasets medianos, correlación no lineal         |
| **IterativeImputer**           |  ✅ (explícita)  |         Alto         | Datasets correlacionados, predictivos            |
| **Multiple Imputation (MICE)** |        ✅✅       |       Muy alto       | Análisis estadístico con incertidumbre medida    |

---

## 5. Recomendación práctica

Si tu foco es **modelado predictivo (ML)**:

* Empieza con `IterativeImputer` (con `BayesianRidge` o `RandomForestRegressor`).
* Evalúa la mejora frente a `SimpleImputer`.

Si tu foco es **análisis estadístico riguroso** (inferencia o papers):

* Usa **Multiple Imputation (MICE)** con librerías como `miceforest` o `autoimpute`.

In [ ]:
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge


In [ ]:
# Imputacion por medio de Simple Imputer
columna_simple_imputer = ['BMI']
columnas_knn_imputer = ['Glucose', 'BloodPressure', 'Outcome']
columnas_iterative_imputer = ['Pregnancies', 'SkinThickness', 'Insulin', ]

In [ ]:
# Identificar patrones de missingness primero
missing_patterns = df.isnull().mean().sort_values(ascending=False)
print("Porcentaje de valores faltantes por columna:")
print(missing_patterns[missing_patterns > 0])


In [ ]:
# Elegir método basado en el análisis
if missing_patterns.max() < 0.05:  # Pocos missing
    imputer = SimpleImputer(strategy='median')
else:  # Patrones complejos
    imputer = IterativeImputer(random_state=42)

# Crear nuevo DataFrame con las mismas columnas
imputed_data = imputer.fit_transform(df)
df_imputed = pd.DataFrame(imputed_data, columns=df.columns, index=df.index)


In [ ]:
generar_distribuciones(df)

In [ ]:
generar_distribuciones(df_imputed)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Cargar tu dataset (asumiendo que ya está cargado como df)
# df = pd.read_csv('data.csv')  # Si necesitas cargarlo

print("Dataset original - Valores faltantes por columna:")
print(df.isnull().sum())

# Definimos cuántas imputaciones queremos

n_imputations = 10
imputed_dfs = []

print(f"\nGenerando {n_imputations} imputaciones múltiples...")

for i in range(n_imputations):

    imputer = IterativeImputer(
        random_state=i, 
        max_iter=10,
        sample_posterior=True  # Muestrea de la distribución posterior
    )

    df_imputed = pd.DataFrame(
        imputer.fit_transform(df), 
        columns=df.columns,
        index=df.index  # Mantener el índice original
    )

    imputed_dfs.append(df_imputed)
    print(f"Imputación {i+1}/{n_imputations} completada")

# Promediamos las versiones imputadas
df_mean = pd.concat(imputed_dfs).groupby(level=0).mean()
df_mean.head()

In [ ]:
generar_distribuciones(df_mean)

In [ ]:

def compare_imputation(original_df, imputed_df, columns_to_compare=None):
    
    if columns_to_compare is None:
        columns_to_compare = original_df.columns
    
    print("COMPARACIÓN DE ESTADÍSTICAS:")
    
    for col in columns_to_compare:
        if col in original_df.select_dtypes(include=[np.number]).columns:
            orig_mean = original_df[col].mean()
            orig_std = original_df[col].std()
            imp_mean = imputed_df[col].mean()
            imp_std = imputed_df[col].std()
            
            print(f"\n{col}:")
            print(f"  Media: {orig_mean:.2f} → {imp_mean:.2f} (Δ: {imp_mean-orig_mean:+.2f})")
            print(f"  Std:   {orig_std:.2f} → {imp_std:.2f} (Δ: {imp_std-orig_std:+.2f})")


In [ ]:
compare_imputation(df, df_mean)

# Ejemplo Simple


#  1. Tipos de mecanismos de datos faltantes

| Tipo                                    | Significado                                                                                        | Ejemplo                                                    | Efecto                                                                                           |
| --------------------------------------- | -------------------------------------------------------------------------------------------------- | ---------------------------------------------------------- | ------------------------------------------------------------------------------------------------ |
| **MCAR** (Missing Completely At Random) | Los valores faltan completamente al azar. No dependen ni de otras variables ni de su propio valor. | Faltan filas por un error de carga o desconexión.          | 🔹 **Pérdida de eficiencia**, pero sin **sesgo**.                                                |
| **MAR** (Missing At Random)             | Los valores faltan en función de **otras variables observadas**, no de la variable faltante.       | Faltan “Job performance” más frecuentemente para IQ bajos. | ⚠️ **Introduce sesgo**, pero puede corregirse con técnicas modernas (IterativeImputer, MICE).    |
| **MNAR** (Missing Not At Random)        | Los valores faltan **por la propia variable faltante**.                                            | Si personas con mal desempeño no reportan su resultado.    | ❌ **Sesgo fuerte**, difícil de corregir (se usan *selection models* o *pattern mixture models*). |


In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    "IQ": [78, 82, 86, 90, 94, 98, 102, 106, 110, 114],
    "JobPerformance": [7.693428, np.nan, np.nan, np.nan, 8.631693, 9.231726, 13.458426, 12.434869, 10.561051, 13.185120]
})

In [ ]:
from sklearn.impute import SimpleImputer

simple_imputer = SimpleImputer(strategy='mean')
df_simple = df.copy()
df_simple['JobPerformance'] = simple_imputer.fit_transform(df[['JobPerformance']])

# Imputa con la **media global**, sin usar IQ.
# No respeta correlación IQ–Performance.

In [ ]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=3)
df_knn = pd.DataFrame(knn_imputer.fit_transform(df), columns=df.columns)

# Usa IQ y JobPerformance juntos para encontrar los vecinos más cercanos y estimar el valor faltante.
# Preserva la correlación de manera **no lineal**

In [ ]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=3)
df_knn = pd.DataFrame(knn_imputer.fit_transform(df), columns=df.columns)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

iter_imputer = IterativeImputer(estimator=BayesianRidge(), random_state=42)
df_iter = pd.DataFrame(iter_imputer.fit_transform(df), columns=df.columns)

# Predice los valores faltantes de JobPerformance en función de IQ mediante regresión bayesiana iterativa.
# Excelente para **datos MAR** con correlación fuerte.


In [ ]:
from sklearn.experimental import enable_iterative_imputer  # habilita el imputer experimental
from sklearn.impute import IterativeImputer

# Definimos cuántas imputaciones queremos
n_imputations = 10
imputed_dfs = []

# Repetimos el proceso con distintas semillas

for i in range(n_imputations):
    imputer = IterativeImputer(random_state=i, max_iter=10)
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    imputed_dfs.append(df_imputed)

# Promediamos las versiones imputadas (solo numéricas)
df_mean = pd.concat(imputed_dfs).groupby(level=0).mean()

df_mean
